In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager
from neuron.neurons.scripted_users import UserConversationalOrchestrator
import os

c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\.venv\Lib\site-packages\accelerate\utils\other.py:220: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  np.core.multiarray._reconstruct,


In [3]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama-3.3-70b-versatile",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = UserConversationalOrchestrator(
    description="You are a human user interacting with conversational and recommendation agents."
)

conversational = Neuron(
    name="conversational",
    llm_config=llm_config,
    system_message="""
    You are a conversational agent.

    Your role is to respond to general user input with brief, friendly, and natural replies.
    You are not responsible for providing recommendations or opinions about movies, products, or any other content.

    Keep your responses simple, casual, and focused only on maintaining a natural conversation.
""",
    description="This agent acts as a social companion and facilitates interactions."
)



llm_config_2 = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

recommendation = Neuron(
    name="recommender",
    llm_config=llm_config_2,
    system_message="""
    You are a movie recommendation agent.

    Your role is to respond with exactly 3 movie suggestions whenever the user asks for a recommendation.

    Always respond directly with the following 3 movies:

    1. The Shawshank Redemption (1994) – A powerful story of hope and friendship set in a prison.
    2. Inception (2010) – A mind-bending sci-fi thriller about dreams within dreams.
    3. Parasite (2019) – A gripping social satire that blends drama, suspense, and dark comedy.

    Do not ask follow-up questions, introduce yourself, apologize, or explain your reasoning.
    Simply return the 3 movies above, and nothing else.
""",
    description="This agent specializes in recommending movies."
)

chitchat = ConversationalOrchestrator(
    agents=[user, conversational, recommendation],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    #send_introductions=True,
    allow_repeat_speaker = False,
)

chitchat_manager = ConversationalOrchestratorManager(chitchat=chitchat, llm_config=llm_config)

In [4]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Hey, how are you doing today?",
    cache=None
)

user_conversational ⟶ conversational_orchestrator:
Hey, how are you doing today?

Next speaker: conversational

conversational ⟶ conversational_orchestrator:
I'm doing well, thanks for asking. How about you, how's your day going so far?

Next speaker: user_conversational

user_conversational ⟶ conversational_orchestrator:
I'm thinking about watching something tonight. Can you recommend me a good movie to relax?

Next speaker: recommender

recommender ⟶ conversational_orchestrator:
1. The Shawshank Redemption (1994) – A powerful story of hope and friendship set in a prison.
2. Inception (2010) – A mind-bending sci-fi thriller about dreams within dreams.
3. Parasite (2019) – A gripping social satire that blends drama, suspense, and dark comedy.

Next speaker: conversational

conversational ⟶ conversational_orchestrator:
Those all sound like great options. What kind of mood are you in tonight? Are you looking for something uplifting, or are you in the mood for something a bit more intense

In [5]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'Hey, how are you doing today?', 'role': 'assistant', 'name': 'user_conversational'}, {'content': "I'm doing well, thanks for asking. How about you, how's your day going so far?", 'name': 'conversational', 'role': 'user'}, {'content': "I'm thinking about watching something tonight. Can you recommend me a good movie to relax?", 'role': 'assistant', 'name': 'user_conversational'}, {'content': '1. The Shawshank Redemption (1994) – A powerful story of hope and friendship set in a prison.\n2. Inception (2010) – A mind-bending sci-fi thriller about dreams within dreams.\n3. Parasite (2019) – A gripping social satire that blends drama, suspense, and dark comedy.', 'name': 'recommender', 'role': 'user'}, {'content': 'Those all sound like great options. What kind of mood are you in tonight? Are you looking for something uplifting, or are you in the mood for something a bit more intense?', 'name': 'conversational', 'role': 'user'}, {'content': '